In [ ]:
import batoid
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
telescope = batoid.Optic.fromYaml("HSC.yaml")

In [ ]:
def pupil(thx, thy, nside=512):
    rays = batoid.RayVector.asGrid(
        optic=telescope, wavelength=750e-9,
        theta_x=thx, theta_y=thy,
        nx=nside, ny=nside
    )
    rays2 = rays.copy()
    telescope.stopSurface.interact(rays2)
    telescope.trace(rays)
    w = ~rays.vignetted
    return rays2.x[w], rays2.y[w]

In [ ]:
def drawCircle(ax, cx, cy, r, **kwargs):
    t = np.linspace(0, 2*np.pi, 1000)
    x = r*np.cos(t)+cx
    y = r*np.sin(t)+cy
    ax.plot(x, y, **kwargs)

In [ ]:
def drawRay(ax, cx, cy, width, theta, **kwargs):
    R = np.array([[np.cos(theta), -np.sin(theta)], [np.sin(theta), np.cos(theta)]])
    
    dx = np.linspace(0, 4.1, 1000)
    dy = np.ones_like(dx)*width/2

    bx = np.copy(dx)
    by = -dy
    
    dx, dy = R.dot(np.vstack([dx, dy]))
    bx, by = R.dot(np.vstack([bx, by]))
    
    dx += cx
    dy += cy
    bx += cx
    by += cy
    
    ax.plot(dx, dy, **kwargs)
    ax.plot(bx, by, **kwargs)

In [ ]:
def drawRectangle(ax, cx, cy, width, height, **kwargs):
    x = width/2*np.array([-1,-1,1,1,-1])
    y = height/2*np.array([-1,1,1,-1,-1])
    x += cx
    y += cy
    ax.plot(x, y, **kwargs)

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
def modelPlot(thx, thy):
    fig, ax = plt.subplots(1, 1, figsize=(30, 30))
    ax.scatter(*pupil(thx,thy), s=0.1, c='k')
    ax.set_aspect('equal')
    # Primary mirror
    drawCircle(ax, 0, 0, 4.1, c='r')
    # Camera shadow
    drawCircle(ax, 17*thx, 17*thy, 0.95, c='r')
    # G1 cutoff
    drawCircle(ax, -142*thx, -142*thy, 4.25, c='r')
    # G4 cutoff
    thr = np.hypot(thx, thy)
    if thr > np.deg2rad(0.75):
        thph = np.arctan2(thy, thx)
        cr = -17.76 - 2e3*(thr-np.deg2rad(0.76)) - 1.8e5*(thr-np.deg2rad(0.76))*(thr-np.deg2rad(0.8))
        drawCircle(ax, cr*np.cos(thph), cr*np.sin(thph), 20.0, c='r')
    
    # spider
    alpha = 6.75

    # TopRing
    trRate = 14
    drawRay(ax, 0.45+trRate*thx, 0.45+trRate*thy, 0.18, np.deg2rad(90+alpha), c='r')
    drawRay(ax, 0.45+trRate*thx, 0.45+trRate*thy, 0.18, np.deg2rad(-alpha), c='r')
    drawRay(ax, -0.45+trRate*thx, -0.45+trRate*thy, 0.18, np.deg2rad(180-alpha), c='r')
    drawRay(ax, -0.45+trRate*thx, -0.45+trRate*thy, 0.18, np.deg2rad(270+alpha), c='r')


    # TertiarySpiderFirstPass
    tsfpRate = 2
    drawRay(ax, 0.45+tsfpRate*thx, 0.45+tsfpRate*thy, 0.18, np.deg2rad(90+alpha), c='r')
    drawRay(ax, 0.45+tsfpRate*thx, 0.45+tsfpRate*thy, 0.18, np.deg2rad(-alpha), c='r')
    drawRay(ax, -0.45+tsfpRate*thx, -0.45+tsfpRate*thy, 0.18, np.deg2rad(180-alpha), c='r')
    drawRay(ax, -0.45+tsfpRate*thx, -0.45+tsfpRate*thy, 0.18, np.deg2rad(270+alpha), c='r')


    # TertiarySpiderSecondPass
    tsspRate = -2
    drawRay(ax, 0.51+tsspRate*thx, 0.51+tsspRate*thy, 0.18, np.deg2rad(90+alpha), c='r')
    drawRay(ax, 0.51+tsspRate*thx, 0.51+tsspRate*thy, 0.18, np.deg2rad(-alpha), c='r')
    drawRay(ax, -0.51+tsspRate*thx, -0.51+tsspRate*thy, 0.18, np.deg2rad(180-alpha), c='r')
    drawRay(ax, -0.51+tsspRate*thx, -0.51+tsspRate*thy, 0.18, np.deg2rad(270+alpha), c='r')

    # FEU
    drawRectangle(ax, 15*thx, 15*thy, 0.84, 2.28, c='r')

    ax.set_xlim(-5,5)
    ax.set_ylim(-5,5)
    ax.axvline(c='k')
    ax.axhline(c='k')
    fig.show()

In [ ]:
telescope.itemDict['SubaruHSC.TopRing'].skip=False
telescope.itemDict['SubaruHSC.BottomRing'].skip=False
telescope.itemDict['SubaruHSC.TertiarySpiderFirstPass'].skip=False
telescope.itemDict['SubaruHSC.TertiarySpiderSecondPass'].skip=False
telescope.itemDict['SubaruHSC.FEU'].skip=False

# modelPlot(0, np.deg2rad(-0.82))
# modelPlot(0, np.deg2rad(-0.77))
# modelPlot(0, np.deg2rad(-0.7))
# modelPlot(0, np.deg2rad(-0.35))
# modelPlot(0, np.deg2rad(0))
# modelPlot(0, np.deg2rad(0.35))
# modelPlot(0, np.deg2rad(0.7))
# modelPlot(0, np.deg2rad(0.77))

# modelPlot(np.deg2rad(-0.77), 0)
# modelPlot(np.deg2rad(-0.7), 0)
# modelPlot(np.deg2rad(-0.35), 0)
# modelPlot(np.deg2rad(0), 0)
# modelPlot(np.deg2rad(0.35), 0)
# modelPlot(np.deg2rad(0.7), 0)
# modelPlot(np.deg2rad(0.77), 0)

# modelPlot(np.deg2rad(-0.5), np.deg2rad(-0.5))
# modelPlot(np.deg2rad(-0.35), np.deg2rad(-0.35))
# modelPlot(np.deg2rad(-0.2), np.deg2rad(-0.2))

# modelPlot(np.deg2rad(0.5), np.deg2rad(-0.5))
# modelPlot(np.deg2rad(0.35), np.deg2rad(-0.35))
# modelPlot(np.deg2rad(0.2), np.deg2rad(-0.2))

modelPlot(np.deg2rad(0.55), np.deg2rad(0.55))